### **Long term Memory Redis**

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_mistralai import ChatMistralAI

from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

import getpass
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Load the API key from the environment
def get_mistral_api_key():
    key = os.getenv("MISTRAL_API_KEY")
    if not key:
        key = getpass.getpass("Enter your Mistral API key: ")
        os.environ["MISTRAL_API_KEY"] = key
    return key

# Get the Mistral API key
mistral_api_key = get_mistral_api_key()

In [5]:
model = ChatMistralAI(model="mistral-small-2501", temperature=0, max_retries=2)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
        ]
)
base_chain = prompt | model

In [ ]:
REDIS_URL = "redis://localhost:6379/0"
store = {}

In [ ]:
def get_message_history(session_id: str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(session_id, url=REDIS_URL)

redis_chain = RunnableWithMessageHistory(
    base_chain,
    get_message_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [ ]:
redis_chain.invoke(
    {"ability": "math", 
     "input": "What does cosine mean?"}, 
    config={"configurable": {"session_id": "math-thread1"}},
)

In [ ]:
redis_chain.invoke(
    {"ability": "math", "input": "Tell me more!"},
    config={"configurable": {"session_id": "math-thread1"}},
)

In [ ]:
redis_chain.invoke(
    {"ability": "physics", 
     "input": "What is the theory of relativity?"}, 
    config={"configurable": {"session_id": "phy-thread1"}},
)

In [ ]:
redis_chain.invoke(
    {"ability": "physics", "input": "Tell me more!"},
    config={"configurable": {"session_id": "phy-thread1"}},
)